## import stuff

In [1]:
from rasa_nlu.model import Metadata, Interpreter
from rasa_nlu.converters import load_data
from rasa_nlu.config import RasaNLUConfig
from rasa_nlu.model import Trainer
import json
#s plit train test
from sklearn.model_selection import train_test_split



In [ ]:
#configs: language, data, model

In [117]:
dir='/mnt/intrctbt/tourism-il-ms/nlp/'
model_directory = dir+'models/model_20170913-193954' #english hotel model
hotel_data_file=dir+'hotel_data.json'
mitie_config=dir+"mitie_config.json"
hotel_train_data_file=dir+'hotel_data_train.json'

## train bot model

In [5]:
training_data = load_data(hotel_data_file)
trainer = Trainer(RasaNLUConfig(mitie_config))

initializing nlp_mitie
initializing tokenizer_mitie
initializing ner_mitie
initializing ner_synonyms
initializing intent_entity_featurizer_regex
initializing intent_classifier_mitie


In [72]:
data = json.loads(training_data.as_json()) #data in human readable format

In [42]:
config = RasaNLUConfig(mitie_config) 

In [116]:
train,test=train_test_split(data['rasa_nlu_data']['common_examples'],random_state=42)
len(train),len(test)

(66, 23)

In [94]:
train_data_to_save={'rasa_nlu_data': {'common_examples':train}}

In [95]:
#save train set to disk
with open(hotel_train_data_file, 'w') as outfile:
    json.dump(train_data_to_save, outfile)

In [97]:
training_data = load_data(hotel_train_data_file)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/rasa_nlu/training_data.py:157: UserWarning: Intent 'hotel-other' has only 1 training examples! minimum is 2, training may fail.
  warnings.warn(template.format(intent, size, self.MIN_EXAMPLES_PER_INTENT))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/rasa_nlu/training_data.py:165: UserWarning: Entity 'pool' has only 1 training examples! minimum is 2, training may fail.
  warnings.warn(template.format(entity, size, self.MIN_EXAMPLES_PER_ENTITY))


In [100]:
trainer = Trainer(RasaNLUConfig(mitie_config))
interpreter=trainer.train(training_data)

initializing nlp_mitie
initializing tokenizer_mitie
initializing ner_mitie
initializing ner_synonyms
initializing intent_entity_featurizer_regex
initializing intent_classifier_mitie
starting to train nlp_mitie
starting to train tokenizer_mitie
starting to train ner_mitie


Exception ignored in: <bound method total_word_feature_extractor.__del__ of <mitie.mitie.total_word_feature_extractor object at 0x7f923239a390>>
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/mitie/mitie.py", line 833, in __del__
    self.__mitie_free(self.__obj)
AttributeError: 'total_word_feature_extractor' object has no attribute '_total_word_feature_extractor__obj'


starting to train ner_synonyms
starting to train intent_entity_featurizer_regex
starting to train intent_classifier_mitie


## check results

In [45]:
metadata = Metadata.load(model_directory)
interpreter = Interpreter.load(metadata, config) #loaded model

In [113]:
len(test)

23

In [110]:
# intet check
sum=0
for item in test:
    hit=interpreter.parse(item['text'])['intent']['name']==item['intent'],item['intent'],\
          interpreter.parse(item['text'])['intent']['name']
    print(hit)
    sum+=hit[0]

(True, 'hotel-search', 'hotel-search')
(True, 'hotel-search', 'hotel-search')
(True, 'live-agent', 'live-agent')
(False, 'affirm', 'hotel-search')
(True, 'hotel-search', 'hotel-search')
(True, 'greet', 'greet')
(True, 'hotel-search', 'hotel-search')
(True, 'affirm', 'affirm')
(True, 'affirm', 'affirm')
(True, 'hotel-search', 'hotel-search')
(True, 'greet', 'greet')
(True, 'live-agent', 'live-agent')
(True, 'hotel-search', 'hotel-search')
(False, 'goodbye', 'hotel-search')
(False, 'goodbye', 'hotel-search')
(True, 'hotel-search', 'hotel-search')
(False, 'goodbye', 'hotel-search')
(True, 'hotel-search', 'hotel-search')
(True, 'hotel-search', 'hotel-search')
(False, 'live-agent', 'goodbye')
(True, 'hotel-search', 'hotel-search')
(True, 'hotel-search', 'hotel-search')
(False, 'live-agent', 'hotel-other')


In [111]:
sum

17

In [114]:
17/23

0.7391304347826086

In [104]:
# entity check accuracy
print('intersect, preds, gts')
num=0
denom=0
for item in test:
    pred=set([ entity['entity'] for entity in interpreter.parse(item['text'])['entities']])
    gt=set([entity['entity'] for entity in item['entities']])
    num+=len(pred.intersection(gt))
    denom+=len(gt)
    print(len(pred.intersection(gt)),len(pred),len(gt))

intersect, preds, gts
1 3 2
2 3 3
0 1 0
0 1 0
3 3 4
0 0 0
1 2 2
0 1 0
0 0 0
2 2 2
0 1 0
0 1 0
2 2 2
0 1 0
0 0 0
2 3 3
0 0 0
1 2 2
2 2 2
0 0 0
1 2 2
2 3 3
0 1 0


In [115]:
# intent accuracy
num/denom

0.7037037037037037